In [1]:
# 문제 2: Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트 (응용편)

import re
from datetime import datetime
from textwrap import dedent
from pprint import pprint
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field

# 환경 설정
load_dotenv()


True

In [2]:

# 1. 도구 정의 (문제 1과 동일하지만 다시 정의)
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])
    
    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()
    
    formatted_docs = [
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
    ]
    
    return formatted_docs

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)
print(llm.model_name)

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm
)

wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)


meta-llama/llama-4-scout-17b-16e-instruct


C:\Users\user\AppData\Local\Temp\ipykernel_8596\4050647688.py:51: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = summary_chain.as_tool(


In [3]:

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for cafe menu-related queries to maintain data confidentiality.
    """
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    cafe_db = FAISS.load_local(
        "./db/cafe_db", 
        embeddings_model, 
        allow_dangerous_deserialization=True
    )
    
    docs = cafe_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]

# 2. Few-shot 예제 정의
examples = [
    HumanMessage("아메리카노의 가격과 특징, 그리고 커피의 역사에 대해 알려주세요.", name="example_user"),
    AIMessage("카페 메뉴를 검색하고, 위키피디아에서 커피 역사 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage("아메리카노: 가격 ₩4,500, 에스프레소와 뜨거운 물로 만든 클래식 블랙 커피, 원두 본연의 맛을 느낄 수 있음", tool_call_id="1"),    
    AIMessage("아메리카노 정보를 찾았습니다. 이제 커피의 역사를 위키피디아에서 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}]),
    ToolMessage("커피는 에티오피아에서 기원하여 아랍을 거쳐 전 세계로 전파된 음료입니다. 15세기 예멘에서 처음 재배되기 시작했으며, 17세기 유럽으로 전해져 커피하우스 문화가 발달했습니다. 산업혁명과 함께 대량 생산이 가능해지면서 현재와 같은 커피 문화가 형성되었습니다.", tool_call_id="2"),
    AIMessage("아메리카노(₩4,500)는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 커피는 에티오피아에서 기원하여 아랍을 거쳐 전 세계로 전파되었으며, 15세기 예멘에서 처음 재배되기 시작했습니다. 17세기 유럽으로 전해져 커피하우스 문화가 발달했고, 산업혁명과 함께 대량 생산이 가능해지면서 현재와 같은 커피 문화가 형성되었습니다.", name="example_assistant"),
]

# 3. Few-shot 프롬프트 템플릿 작성
today = datetime.today().strftime("%Y-%m-%d")

system = """You are an AI assistant providing cafe menu information and general food/beverage-related knowledge.
For information about the cafe's menu, use the db_search_cafe_func tool.
For other general information about food, beverages, and their history, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the tavily_search_func tool.

Guidelines:
1. For cafe menu queries, always search the menu database first
2. For general knowledge about food/beverages, use Wikipedia
3. For latest trends or news, use web search
4. Provide comprehensive answers by combining multiple sources when needed
5. Always cite the source of your information
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f" Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# 4. 도구를 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

# 5. Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 6. 도구 실행 결과를 종합하여 최종 답변을 생성하는 체인 구현
@chain
def advanced_cafe_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)
    
    print("AI가 선택한 도구들:")
    for tool_call in ai_msg.tool_calls:
        print(f"- {tool_call['name']}: {tool_call['args']}")
    print("-" * 50)
    
    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"실행 중: {tool_call['name']}")
        
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
    
    print(f"총 {len(tool_msgs)}개의 도구 결과를 받았습니다.")
    print("-" * 50)
    
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [4]:

# 7. 실행 및 테스트
if __name__ == "__main__":
    # 복합 질문 테스트
    query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
    
    print(f"질문: {query}")
    print("=" * 80)
    
    response = advanced_cafe_chain.invoke(query)
    
    print("최종 답변:")
    print(response.content)
    
    # 추가 테스트 질문들
    additional_queries = [
        "콜드브루의 가격과 특징을 알려주세요.",
        "티라미수와 잘 어울리는 커피는 무엇인가요? 티라미수의 유래도 설명해주세요.",
        "최근 커피 트렌드는 무엇인가요?"
    ]
    
    print("\n" + "=" * 80)
    print("추가 테스트 질문들:")
    
    for i, additional_query in enumerate(additional_queries, 1):
        print(f"\n{i}. {additional_query}")
        try:
            additional_response = advanced_cafe_chain.invoke(additional_query)
            print(f"답변: {additional_response.content[:200]}...")  # 처음 200자만 출력
        except Exception as e:
            print(f"오류 발생: {e}")


질문: 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.
AI가 선택한 도구들:
- db_search_cafe_func: {'query': '카페라둥'}
--------------------------------------------------
실행 중: db_search_cafe_func
총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
최종 답변:
카라멜 마키아토와 바닐라 라떼가 카페라떼와 잘 어울리는 디저트입니다. 이제 라떼의 유래를 알려드리겠습니다.

추가 테스트 질문들:

1. 콜드브루의 가격과 특징을 알려주세요.
AI가 선택한 도구들:
- db_search_cafe_func: {'query': '콰드부룰'}
--------------------------------------------------
실행 중: db_search_cafe_func
총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
답변: 콜드브루: 가격 ₩5,000, 찬물에 12-24시간 우려낸 콜드브루 원액으로 만든 시원한 커피, 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있음...

2. 티라미수와 잘 어울리는 커피는 무엇인가요? 티라미수의 유래도 설명해주세요.
AI가 선택한 도구들:
- wiki_summary: {'query': '티라미수 유래'}
--------------------------------------------------
실행 중: wiki_summary
총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
답변: 티라미수는 이탈리아 디저트로, 커피에 적신 레이디핑거를 마스카포네 치즈 크림과 코코아 가루로 층을 이루어 만든 dessert입니다. 이름은 이탈리아어로 "나를 들어 올려"

C:\Users\user\AppData\Local\Temp\ipykernel_8596\4050647688.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)


총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
답변: 2024년 커피 트렌드는 '도파밍'과 '버라이어티 가격 전략' 등이 주목받고 있습니다. '도파밍'은 도파민을 추구하는 트렌드로, 커피의 풍미와 카페인의 효과를 즐기는 소비자들이 증가하고 있습니다. '버라이어티 가격 전략'은 다양한 가격대의 커피를 제공하는 전략으로, 소비자들의 다양한 취향과 예산에 맞춰 선택할 수 있는 옵션을 제공하는 것입니다....


In [5]:

# 8. Gradio UI 구현 (선택사항)
def create_gradio_interface():
    """
    Gradio를 사용한 웹 인터페이스 생성
    """
    import gradio as gr
    from typing import List, Tuple
    
    def chat_interface(message: str, history: List[Tuple[str, str]]) -> str:
        try:
            # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
            chat_history = []
            for human, ai in history[-2:]:  # 최근 2개의 대화만 사용
                chat_history.append(HumanMessage(content=human))
                chat_history.append(AIMessage(content=ai))
            
            # 체인 실행
            response = advanced_cafe_chain.invoke(message)
            return response.content
            
        except Exception as e:
            return f"죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다: {str(e)}"
    
    # 예제 질문들
    example_questions = [
        "아메리카노의 가격과 특징을 알려주세요.",
        "카페라떼와 어울리는 디저트는 무엇인가요?",
        "콜드브루의 유래에 대해 설명해주세요.",
        "최근 커피 트렌드를 검색해주세요."
    ]
    
    # Gradio 인터페이스 생성
    demo = gr.ChatInterface(
        fn=chat_interface,
        title="카페 메뉴 AI 어시스턴트",
        description="카페 메뉴 정보, 음료 추천, 음식 관련 질문에 답변해 드립니다.",
        examples=example_questions,
        theme=gr.themes.Soft()
    )
    
    return demo

# 실행 예시
if __name__ == "__main__":
    # 기본 콘솔 테스트
    print("=== 카페 AI 어시스턴트 테스트 ===")
    
    # 웹 인터페이스 실행 (선택사항)
    demo = create_gradio_interface()
    demo.launch()
    
    print("테스트 완료!")

=== 카페 AI 어시스턴트 테스트 ===


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.13\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


테스트 완료!
